In [1]:
import pickle
import pandas as pd
import os           
import numpy as np
import re
import itertools
import plotly.express as px
#import brightway2 as bw 
import random 
import string

In [2]:
import sys
sys.path.append('../../Module')  #two levels up & then down to Module folder
#from recipe_mod import *
from recipe_mod_resIC import *

<h2> raw RIVM has only 1 EF, and BW2 with 12 EFs, same CFs compared to SP/oLCA </h2>

water inflow = 1, outflow = -1
<h3> pairwise_comp only for SP vs. oLCA </h3>

In [3]:
lcia_name = "Water consumption"
SP_name = "Water consumption"
rivm_name = "Water consumption"
olca_filepathname = "../../Data_rawCFs/olca_recipe_water_saved.dat"
bw2_filepathname = "../../Data_rawCFs/BW2_recipe_Water consumption.dat"
# used in final calculation to create dict names
pairwise_name = ["SPvsRIVM", "SPvsBW2", "SPvsopenLCA", "openLCAvsBW2", "openLCAvsRIVM", "BW2vsRIVM"]

### read in SP LCIA

In [4]:
SP_recipe = pd.read_excel("../../Data_rawCFs/SP_ReCiPe.xlsx", sheet_name=SP_name, 
                                  header=[0], converters={'CAS':str})
print(len(SP_recipe))
SP_recipe.fillna(''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(9)), inplace=True)
#to assign a random string_num to CAS with NA, so that they're not the "SAME" CAS when compared to other sources:
#print ([x for x in SP_recipe["CAS"] if '-' not in x ] [0]) 
SP_recipe.columns

2091


Index(['Compartment', 'Subcompartment', 'Substance', 'CAS', 'Factor', 'Unit'], dtype='object')

In [5]:
SP_recipe = SP_recipe.rename(columns={"Compartment": "category", "Subcompartment": "subcategory", 
                               "Substance": "flow", "Factor":"value", "CAS": "cas_number" })
SP_recipe = SP_recipe.rename(str.lower, axis='columns')

In [6]:
SP_recipe['flow'] = SP_recipe["flow"].str.lower()
SP_recipe.head()

,category,subcategory,flow,cas_number,value,unit
0,Raw,(unspecified),"water, barrage",007732-18-5,0.001,m3 / kg
1,Raw,(unspecified),"water, cooling, drinking",007732-18-5,0.001,m3 / kg
2,Raw,(unspecified),"water, cooling, surface",007732-18-5,0.001,m3 / kg
3,Raw,(unspecified),"water, cooling, unspecified natural origin, ad",007732-18-5,1.000,m3 / m3
4,Raw,(unspecified),"water, cooling, unspecified natural origin, ae",007732-18-5,1.000,m3 / m3


### read in openlca LCIA

In [7]:
olca_recipe = unpickle_data(olca_filepathname)
olca_recipe.fillna(''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(9)), inplace=True)
#print ([x for x in olca_recipe["CAS_NUMBER"] if '-' not in x ] [0]) 
print(len(olca_recipe))
olca_recipe = olca_recipe.rename(str.lower, axis='columns')

4727


In [8]:
olca_recipe['flow'] = olca_recipe["flow"].str.lower()
olca_recipe.head()

,impact_method,impact_category_uuid,impact_category,reference_unit,flow_uuid,flow,cas_number,subcategory,category,unit,value
0,ReCiPe 2016 Midpoint (H),95d88b64-0767-321b-8f02-9e3e7c65a527,Water consumption,m3,2ec98519-0441-3879-bc20-0483a724fab1,"water, unspecified natural origin, row",R3J9WFUUY,in ground,Resource,m3,1.0
1,ReCiPe 2016 Midpoint (H),95d88b64-0767-321b-8f02-9e3e7c65a527,Water consumption,m3,abc005bc-18ef-4134-a532-215fe609b66e,"water, unspecified natural origin, ga",R3J9WFUUY,in ground,Resource,m3,1.0
2,ReCiPe 2016 Midpoint (H),95d88b64-0767-321b-8f02-9e3e7c65a527,Water consumption,m3,c3b30b05-89b9-4429-afdd-f2555ba9cb14,"water, unspecified natural origin, sm",R3J9WFUUY,in ground,Resource,m3,1.0
3,ReCiPe 2016 Midpoint (H),95d88b64-0767-321b-8f02-9e3e7c65a527,Water consumption,m3,39a69e8b-8315-4cbd-95b7-7b38a4570683,"water, unspecified natural origin, pg",R3J9WFUUY,in ground,Resource,m3,1.0
4,ReCiPe 2016 Midpoint (H),95d88b64-0767-321b-8f02-9e3e7c65a527,Water consumption,m3,7a576055-2843-4b8c-8d52-cdf2b4c5af3a,"water, unspecified natural origin, lt",R3J9WFUUY,in ground,Resource,m3,1.0


### read in BW2 LCIA 

In [9]:
bw_recipe = unpickle_data(bw2_filepathname)
bw_recipe.fillna(''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(9)), inplace=True)
#print ([x for x in olca_recipe["CAS_NUMBER"] if '-' not in x ] [0]) 
print(len(bw_recipe))
bw_recipe.columns  #already named in the right format and lower.str : flow, cas_number

12


Index(['cas_number', 'flow', 'flow_uuid', 'type', 'category', 'subcategory',
       'unit', 'value'],
      dtype='object')

In [10]:
bw_recipe['flow'] = bw_recipe["flow"].str.lower()
bw_recipe.head()

,cas_number,flow,flow_uuid,type,category,subcategory,unit,value
0,007732-18-5,water,2404b41a-2eed-4e9d-8ab6-783946fdf5d6,emission,water,unspecified,cubic meter,-1
1,007732-18-5,water,2256a142-8242-4b4f-b9aa-a167803989ca,emission,water,fossil well,cubic meter,-1
2,007732-18-5,water,51254820-3456-4373-b7b4-056cf7b16e01,emission,water,ground-,cubic meter,-1
3,007732-18-5,water,06d4812b-6937-4d64-8517-b69aabce3648,emission,water,"ground-, long-term",cubic meter,-1
4,007732-18-5,water,db4566b1-bd88-427d-92da-2d25879063b9,emission,water,surface water,cubic meter,-1


### read in raw RIVM_ReCiPe 

In [11]:
rivm_recipe = pd.read_excel("../../Data_rawCFs/ReCiPe2016_CFs_v1.1_20180117.xlsx", sheet_name=rivm_name, 
                                  header=[1], converters={'CAS':str})
rivm_recipe

,Unnamed: 0,Individualist,Hierarchist,Egalitarian
0,Water consumption,1,1,1


In [12]:
rivm_recipe = rivm_recipe.rename(columns={"Unnamed: 0": "flow", "Hierarchist":"value"})
rivm_recipe

,flow,Individualist,value,Egalitarian
0,Water consumption,1,1,1


### 3. final calculation, pair-wise comparison, corr. matrix

In [13]:
# create 6 empty dict: dict_SPvsRIVM; dict_SPvsBW2; dict_SPvsopenLCA; dict_openLCAvsBW2; dict_openLCAvsRIVM; dict_BW2vsRIVM
# each dict with five keys and null values (to be assigned later)
for name in pairwise_name:
    #print(name)
    keys = [name+"_commonEF_sumtable", name+"_commonEF_list", name + "_result_diffEFs", name + "_result_corr" ]
    globals()['dict_%s' % name] = dict.fromkeys(keys, None)

In [14]:
k_list = list(dict_SPvsRIVM)
dict_SPvsRIVM[k_list[0]], dict_SPvsRIVM[k_list[1]] = res_compare_included_EF(SP_recipe, "flow", rivm_recipe,  "flow", to_print = "no" )
if len(dict_SPvsRIVM[k_list[1]] ) == 0: 
    dict_SPvsRIVM[k_list[2]] = pd.DataFrame ({'SP_value': np.NaN , 'RIVM_value': np.NaN }, index=[0])
else:
    dict_SPvsRIVM[k_list[2]] = pd.DataFrame (
        {'SP_value': final_EF_combined(SP_recipe, "flow", "value", rivm_recipe, "flow", "value", dict_SPvsRIVM[k_list[1]])[0] , 
         'RIVM_value': final_EF_combined(SP_recipe, "flow", "value", rivm_recipe, "flow", "value", dict_SPvsRIVM[k_list[1]])[1] }, 
        index = dict_SPvsRIVM[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 0


In [15]:
k_list = list(dict_SPvsBW2)
dict_SPvsBW2[k_list[0]],dict_SPvsBW2[k_list[1]] = res_compare_included_EF(SP_recipe, "flow", bw_recipe,  "flow", to_print = "no" )
if len(dict_SPvsBW2[k_list[1]] ) == 0: 
    dict_SPvsBW2[k_list[2]] = pd.DataFrame ({'SP_value': np.NaN , 'BW2_value': np.NaN }, index=[0])
else:
    dict_SPvsBW2[k_list[2]] = pd.DataFrame (
        {'SP_value': final_EF_combined(SP_recipe, "flow", "value", bw_recipe, "flow", "value", dict_SPvsBW2[k_list[1]])[0] , 
         'BW2_value': final_EF_combined(SP_recipe, "flow", "value", bw_recipe, "flow", "value", dict_SPvsBW2[k_list[1]])[1] }, 
        index = dict_SPvsBW2[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 4
Different value for a same flow: water
Different value for a same flow: water


<font color='red'>for water, diff. CF value for the same EF (becaz diff. type (emission/resource) and diff. compartment: ocean = 0, freshwater = 1), for simplicity, they will be assigned nan value for corr. matrix </font>


In [16]:
k_list = list(dict_SPvsopenLCA)
dict_SPvsopenLCA[k_list[0]],dict_SPvsopenLCA[k_list[1]] = res_compare_included_EF(SP_recipe, "flow", olca_recipe,  "flow", to_print = "no" )
if len(dict_SPvsopenLCA[k_list[1]] ) == 0: 
    dict_SPvsopenLCA[k_list[2]] = pd.DataFrame ({'SP_value': np.NaN , 'olca_value': np.NaN }, index=[0])
else:
    dict_SPvsopenLCA[k_list[2]] = pd.DataFrame (
        {'SP_value': final_EF_combined(SP_recipe, "flow", "value", olca_recipe, "flow", "value", dict_SPvsopenLCA[k_list[1]])[0] , 
         'olca_value': final_EF_combined(SP_recipe, "flow", "value", olca_recipe, "flow", "value", dict_SPvsopenLCA[k_list[1]])[1] },
        index = dict_SPvsopenLCA[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 1820
Different value for a same flow: water, as
Different value for a same flow: water, kn
Different value for a same flow: water, rer
Different value for a same flow: water, je
Different value for a same flow: water, za
Different value for a same flow: water, sb
Different value for a same flow: water, iai area, eu27 & efta
Different value for a same flow: water, km
Different value for a same flow: water, oce
Different value for a same flow: water, dk
Different value for a same flow: water, mw
Different value for a same flow: water, mn
Different value for a same flow: water, tz
Different value for a same flow: water, gt
Different value for a same flow: water, ua
Different value for a same flow: water, ec
Different value for a same flow: water, cs
Different value for a same flow: water, hn
Different value for a same flow: water, es
Different value for a same flow: 

Different value for a same flow: water, ee
Different value for a same flow: water, row
Different value for a same flow: water, zm
Different value for a same flow: water, extreme water stress
Different value for a same flow: water, hr
Different value for a same flow: water, nz
Different value for a same flow: water, tm
Different value for a same flow: water, sn
Different value for a same flow: water, lu
Different value for a same flow: water, ly
Different value for a same flow: water, bn
Different value for a same flow: water, iai area, south america
Different value for a same flow: water, ma
Different value for a same flow: water, ar
Different value for a same flow: water, ng
Different value for a same flow: water, bw
Different value for a same flow: water, np
Different value for a same flow: water, lc
Different value for a same flow: water, eg
Different value for a same flow: water, tg
Different value for a same flow: water, ad
Different value for a same flow: water, ht
Different valu

Different value for a same flow: water, europe without switzerland
Different value for a same flow: water, kp
Different value for a same flow: water, kr
Different value for a same flow: water, is
Different value for a same flow: water, rla
Different value for a same flow: water, weu
Different value for a same flow: water, gi
Different value for a same flow: water, rna
Different value for a same flow: water, mg
Different value for a same flow: water, ni
Different value for a same flow: water, id
Different value for a same flow: water, europe, without russia and turkey
Different value for a same flow: water, sg
Different value for a same flow: water, aw
Different value for a same flow: water, ml
Different value for a same flow: water, pl
Different value for a same flow: water, sj
Different value for a same flow: water, gl
Different value for a same flow: water, iai area, north america, without quebec
Different value for a same flow: water, jp
Different value for a same flow: water, se
Di

Different value for a same flow: water, au
Different value for a same flow: water, bb
Different value for a same flow: water, er
Different value for a same flow: water, hk
Different value for a same flow: water, in
Different value for a same flow: water, eh
Different value for a same flow: water, ne
Different value for a same flow: water, mk
Different value for a same flow: water, cx
Different value for a same flow: water, dj
Different value for a same flow: water, no
Different value for a same flow: water, al
Different value for a same flow: water, il
Different value for a same flow: water, ye
Different value for a same flow: water, bm
Different value for a same flow: water, om
Different value for a same flow: water, dm
Different value for a same flow: water, jm
Different value for a same flow: water, mt
Different value for a same flow: water, pg
Different value for a same flow: water, gr
Different value for a same flow: water, tn
Different value for a same flow: water, rme
Different 

Different value for a same flow: water, bt
Different value for a same flow: water, cf
Different value for a same flow: water, ga
Different value for a same flow: water, pf
Different value for a same flow: water, cz
Different value for a same flow: water, iai area, africa
Different value for a same flow: water, pa
Different value for a same flow: water, ucte
Different value for a same flow: water, bz
Different value for a same flow: water, tl
Different value for a same flow: water, ras
Different value for a same flow: water, cg
Different value for a same flow: water, raf
Different value for a same flow: water, mo
Different value for a same flow: water, na
Different value for a same flow: water, mr
Different value for a same flow: water, li
Different value for a same flow: water, be
Different value for a same flow: water, tv
Different value for a same flow: water, moderate water stress
Different value for a same flow: water, lv
Different value for a same flow: water, low water stress
Dif

Different value for a same flow: water, io
Different value for a same flow: water, cd
Different value for a same flow: water, tw
Different value for a same flow: water, im
Different value for a same flow: water, gs
Different value for a same flow: water, vn
Different value for a same flow: water, gh
Different value for a same flow: water, rs
Different value for a same flow: water, fj
Different value for a same flow: water, gb
Different value for a same flow: water, tt
Different value for a same flow: water, nf
Different value for a same flow: water, fo
Different value for a same flow: water, me
Different value for a same flow: water, iai area, asia, without china and gcc
Different value for a same flow: water, de
Different value for a same flow: water, md
Different value for a same flow: water, rw
Different value for a same flow: water, mh
Different value for a same flow: water, ke
Different value for a same flow: water, fm
Different value for a same flow: water, ee
Different value for

In [17]:
k_list = list(dict_openLCAvsBW2)
dict_openLCAvsBW2[k_list[0]],dict_openLCAvsBW2[k_list[1]] = res_compare_included_EF(olca_recipe, "flow_uuid", bw_recipe, "flow_uuid", to_print = "no" )

if len(dict_openLCAvsBW2[k_list[1]] ) == 0: 
    dict_openLCAvsBW2[k_list[2]] = pd.DataFrame ({'olca_value': np.NaN , 'BW2_value': np.NaN }, index=[0])
else:
    dict_openLCAvsBW2[k_list[2]] = pd.DataFrame (
        {'olca_value': final_EF_combined(olca_recipe, "flow_uuid", "value", bw_recipe, "flow_uuid", "value", 
                                       dict_openLCAvsBW2[k_list[1]])[0] , 
         'BW2_value': final_EF_combined(olca_recipe, "flow_uuid", "value", bw_recipe, "flow_uuid", "value",
                                         dict_openLCAvsBW2[k_list[1]])[1] },
        index =  dict_openLCAvsBW2[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 10


In [18]:
k_list = list(dict_openLCAvsRIVM)
dict_openLCAvsRIVM[k_list[0]],dict_openLCAvsRIVM[k_list[1]] = res_compare_included_EF(olca_recipe, "flow", rivm_recipe, "flow", to_print = "no" )

if len(dict_openLCAvsRIVM[k_list[1]] ) == 0: 
    dict_openLCAvsRIVM[k_list[2]] = pd.DataFrame ({'olca_value': np.NaN , 'RIVM_value': np.NaN }, index=[0])
else:
    dict_openLCAvsRIVM[k_list[2]] = pd.DataFrame (
        {'olca_value': final_EF_combined(olca_recipe, "flow", "value", rivm_recipe, "flow", "value", 
                                       dict_openLCAvsRIVM[k_list[1]])[0] , 
         'BW2_value': final_EF_combined(olca_recipe, "flow", "value", rivm_recipe, "flow", "value",
                                         dict_openLCAvsRIVM[k_list[1]])[1] },
        index = dict_openLCAvsRIVM[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 0


In [19]:
k_list = list(dict_BW2vsRIVM)
dict_BW2vsRIVM[k_list[0]],dict_BW2vsRIVM[k_list[1]] = res_compare_included_EF(bw_recipe, "flow", rivm_recipe, "flow", to_print = "no" )
if len(dict_BW2vsRIVM[k_list[1]] ) == 0: 
    dict_BW2vsRIVM[k_list[2]] = pd.DataFrame ({'BW2_value': np.NaN , 'RIVM_value': np.NaN }, index=[0])
else:
    dict_BW2vsRIVM[k_list[2]] = pd.DataFrame (
        {'BW2_value': final_EF_combined(bw_recipe, "flow", "value", rivm_recipe, "flow", "value", 
                                       dict_BW2vsRIVM[k_list[1]])[0] , 
         'RIVM_value': final_EF_combined(bw_recipe, "flow", "value", rivm_recipe, "flow", "value",
                                         dict_BW2vsRIVM[k_list[1]])[1] },
        index = dict_BW2vsRIVM[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 0


In [20]:
for name in pairwise_name:
    new_df = globals()['dict_%s' % name][name + "_result_diffEFs"]
    try:
        new_df_corr = new_df.corr()
    except: 
        error
    print(new_df_corr)
    globals()['dict_%s' % name][name + "_result_corr"] = new_df_corr

            SP_value  RIVM_value
SP_value         NaN         NaN
RIVM_value       NaN         NaN
           SP_value  BW2_value
SP_value        NaN        NaN
BW2_value       NaN        1.0
            SP_value  olca_value
SP_value         1.0         1.0
olca_value       1.0         1.0
            olca_value  BW2_value
olca_value         1.0        1.0
BW2_value          1.0        1.0
            olca_value  RIVM_value
olca_value         NaN         NaN
RIVM_value         NaN         NaN
            BW2_value  RIVM_value
BW2_value         NaN         NaN
RIVM_value        NaN         NaN


In [21]:
# saving each dict_ to a single worksheet, different rows , # saving to sub-folder "results/Resources_IC/"

if not os.path.exists("results/Resources_IC"):
    os.makedirs("results/Resources_IC")

filename = lcia_name + "_pairwise_comp_result.xlsx"
writer = pd.ExcelWriter("results/Resources_IC/" + filename, engine='xlsxwriter')

for name in pairwise_name:
    sum_table = globals()['dict_%s' % name][name + "_commonEF_sumtable"]
    new_df = globals()['dict_%s' % name][name + "_result_diffEFs"]
    new_df_corr = globals()['dict_%s' % name][name + "_result_corr"]

    sum_table.to_excel(writer,sheet_name=name,startrow=0 , startcol=0)   # write first  (3rows)
    new_df_corr.to_excel(writer,sheet_name=name,startrow=5 , startcol=0) # write next   (3rows)
    new_df.to_excel(writer,sheet_name=name,startrow=10 , startcol=0)     # finally, write the resulting EFs 
     
writer.save()